# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [12]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [13]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [14]:
print("There is {} event files and the first 5 files are: {}".format(len(file_path_list), file_path_list[:5]))

There is 30 event files and the first 5 files are: ['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
df = pd.read_csv("event_data/2018-11-30-events.csv")

In [16]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540990e+12,829,Jim Henson's Dead,200,1.543540e+12,91.0
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540560e+12,1049,Shell Shock,200,1.543540e+12,73.0
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540560e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1.543540e+12,73.0
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540560e+12,1049,Won't Go Home Without You,200,1.543540e+12,73.0
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540560e+12,1049,Hey_ Soul Sister,200,1.543540e+12,73.0


In [17]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [18]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

<b>Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. The event_datafile_new.csv contains the following columns: </b>
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:

<img src="images/image_event_datafile_new.jpg">

In [27]:
# read the data with pandas
df = pd.read_csv("event_datafile_new.csv")

In [29]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [21]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [23]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

Now we need to create tables to run the following queries.  
Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [53]:
# first check for duplicates in the key (userId, sessionId, itemInSession)
assert sum(df[["sessionId", "itemInSession"]].duplicated()) == 0

Ok so there is no duplicated rows when selecting (<i>sessionId</i>, <i>itemInSession</i>) columns and these columns can be used as key for the table

In [61]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history 
## that was heard during sessionId = 338, and itemInSession = 4

create_session = """
    CREATE TABLE IF NOT EXISTS session_table (
        user_id INT,
        session_id INT,
        item_in_session INT,
        artist_name TEXT,
        song_title TEXT,
        song_length FLOAT,
        PRIMARY KEY (session_id, item_in_session)
    )
"""

try:
    session.execute("DROP TABLE IF EXISTS session_table")
    session.execute(create_session)
except Exception as e:
    print(e)

In [62]:
df.iloc[:,[10,8,3,0,9,5]].head()

,userId,sessionId,itemInSession,artist,song,length
0,91,829,0,Stephen Lynch,Jim Henson's Dead,182.85669
1,73,1049,0,Manowar,Shell Shock,247.56200
2,73,1049,1,Morcheeba,Women Lose Weight (Feat: Slick Rick),257.41016
3,73,1049,2,Maroon 5,Won't Go Home Without You,231.23546
4,73,1049,3,Train,Hey_ Soul Sister,216.76363


In [63]:
df.iloc[:,[10,8,3,0,9,5]].dtypes

userId             int64
sessionId          int64
itemInSession      int64
artist            object
song              object
length           float64
dtype: object

In [64]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO session_table (
                user_id ,session_id, item_in_session, artist_name, song_title, song_length
            )
        """
        query = query + "VALUES (%s ,%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

<b>Do a SELECT to verify that the data have been inserted into each table</b>

In [65]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist_name, song_title, song_length
    FROM session_table
    WHERE session_id=338
        AND item_in_session=4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


## QUERY 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [60]:
# first check for duplicates in the key (userId, sessionId, itemInSession)
assert sum(df[["userId", "sessionId", "itemInSession"]].duplicated()) == 0

Ok so there is no duplicated rows when selecting (<i>userId</i>, <i>sessionId</i>, <i>itemInSession</i>) columns and these columns can be used as key for the table

In [88]:
## TO-DO: Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

create_user = """
    CREATE TABLE IF NOT EXISTS user_table (
        user_id INT,
        user_first_name TEXT,
        user_last_name TEXT,
        session_id INT,
        item_in_session INT,
        artist_name TEXT,
        song_title TEXT,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
"""

try:
    session.execute("DROP TABLE IF EXISTS user_table")
    session.execute(create_user)
except Exception as e:
    print(e)

In [89]:
df.iloc[:,[10,1,4,8,3,0,9]].head()

,userId,firstName,lastName,sessionId,itemInSession,artist,song
0,91,Jayden,Bell,829,0,Stephen Lynch,Jim Henson's Dead
1,73,Jacob,Klein,1049,0,Manowar,Shell Shock
2,73,Jacob,Klein,1049,1,Morcheeba,Women Lose Weight (Feat: Slick Rick)
3,73,Jacob,Klein,1049,2,Maroon 5,Won't Go Home Without You
4,73,Jacob,Klein,1049,3,Train,Hey_ Soul Sister


In [90]:
df.iloc[:,[10,1,4,8,3,0,9]].dtypes

userId            int64
firstName        object
lastName         object
sessionId         int64
itemInSession     int64
artist           object
song             object
dtype: object

In [91]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO user_table (
                user_id, user_first_name, user_last_name, session_id, item_in_session, artist_name, song_title
            )
        """
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), str(line[1]), str(line[4]), int(line[8]), int(line[3]), str(line[0]), str(line[9])))

<b> Do a SELECT to verify that the data have been inserted into each table</b>

In [92]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist_name, song_title, user_first_name, user_last_name
    FROM user_table
    WHERE user_id=10
        AND session_id=182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


## QUERY 3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [183]:
## TO-DO: Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

#create_song = """
#    CREATE TABLE IF NOT EXISTS song_table (
#        song_title TEXT,
#        user_id INT,
#        user_first_name TEXT,
#        user_last_name TEXT,
#        PRIMARY KEY ((song_title, user_id))
#    )
#"""

create_song = """
    CREATE TABLE IF NOT EXISTS song_table (
        user_id INT,
        user_first_name TEXT,
        user_last_name TEXT,
        song_title TEXT,
        PRIMARY KEY (song_title, user_id)
    )
"""

try:
    session.execute("DROP TABLE IF EXISTS song_table")
    session.execute(create_song)
except Exception as e:
    print(e)

In [185]:
df.iloc[:,[10,1,4,9]].head()

,userId,firstName,lastName,song
0,91,Jayden,Bell,Jim Henson's Dead
1,73,Jacob,Klein,Shell Shock
2,73,Jacob,Klein,Women Lose Weight (Feat: Slick Rick)
3,73,Jacob,Klein,Won't Go Home Without You
4,73,Jacob,Klein,Hey_ Soul Sister


In [187]:
df.iloc[:,[10,1,4,9]].dtypes

userId        int64
firstName    object
lastName     object
song         object
dtype: object

In [188]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        #query = """
        #    INSERT INTO song_table (
        #        song_title, user_id, user_first_name, user_last_name
        #    )
        #"""
        query = """
            INSERT INTO song_table (
                user_id,
                user_first_name,
                user_last_name,
                song_title
            )
        """
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), str(line[1]), str(line[4]), str(line[9])))

<b> Do a SELECT to verify that the data have been inserted into each table</b>

In [189]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    SELECT user_first_name, user_last_name
    FROM song_table
    WHERE song_title='All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [193]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS session_table")
    session.execute("DROP TABLE IF EXISTS user_table")
    session.execute("DROP TABLE IF EXISTS song_table")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [194]:
session.shutdown()
cluster.shutdown()